### Reading In Files 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import matplotlib.ticker as ticker
import plotly.express as px


In [ ]:
# df_trimester1=pd.read_csv(r"C:\Users\R I B\Desktop\PFE\inventory_trimester_1.csv")
# df_trimester2=pd.read_csv(r"C:\Users\R I B\Desktop\PFE\inventory_trimester_2.csv")
# df_trimester3=pd.read_csv(r"C:\Users\R I B\Desktop\PFE\inventory_trimester_3.csv")
# df_trimester4=pd.read_csv(r"C:\Users\R I B\Desktop\PFE\inventory_trimester_4.csv")
df_trimester1=pd.read_csv(r"C:\Users\alaak\Downloads\inventory_semester_1.csv")
df_trimester2=pd.read_csv(r"C:\Users\alaak\Downloads\inventory_semester_2.csv")
df_trimester3=pd.read_csv(r"C:\Users\alaak\Downloads\inventory_semester_3.csv")
df_trimester4=pd.read_csv(r"C:\Users\alaak\Downloads\inventory_semester_4.csv")


### Check for Duplicates

In [ ]:
print(df_trimester1.duplicated().sum())
print(df_trimester2.duplicated().sum())
print(df_trimester3.duplicated().sum())
print(df_trimester4.duplicated().sum())


### Remove Duplicates

In [ ]:
df_trimester1=df_trimester1.drop_duplicates()
df_trimester2=df_trimester2.drop_duplicates()
df_trimester3=df_trimester3.drop_duplicates()
df_trimester4=df_trimester4.drop_duplicates()
print(df_trimester1.duplicated().sum())
print(df_trimester2.duplicated().sum())
print(df_trimester3.duplicated().sum())
print(df_trimester4.duplicated().sum())

### Merge The Data Frames

In [ ]:
df=pd.concat([df_trimester1, df_trimester2, df_trimester3, df_trimester4], axis=0)

### Check the first few rows

In [ ]:
df.head()

### Get the Summary of The DataFrame

In [ ]:
df.info()

### Check the Nulls

In [ ]:
print(df.isnull().sum())

### Check the sold_quantity,returned_quantity,damaged_quantity Columns

In [ ]:
print(df["sold_quantity"].value_counts(dropna=False))

In [ ]:
print(df["returned_quantity"].value_counts(dropna=False))


In [ ]:
print(df["damaged_quantity"].value_counts(dropna=False))

### Replace "unknown" with NaN

In [ ]:
df_trimester1[["sold_quantity", "returned_quantity", "damaged_quantity"]] = df_trimester1[["sold_quantity", "returned_quantity", "damaged_quantity"]].replace(["unknown"," ",0], np.nan)
df_trimester2[["sold_quantity", "returned_quantity", "damaged_quantity"]] = df_trimester2[["sold_quantity", "returned_quantity", "damaged_quantity"]].replace(["unknown"," ",0], np.nan)
df_trimester3[["sold_quantity", "returned_quantity", "damaged_quantity"]] = df_trimester3[["sold_quantity", "returned_quantity", "damaged_quantity"]].replace(["unknown"," ",0], np.nan)
df_trimester4[["sold_quantity", "returned_quantity", "damaged_quantity"]] = df_trimester4[["sold_quantity", "returned_quantity", "damaged_quantity"]].replace(["unknown"," ",0], np.nan)
df[["sold_quantity", "returned_quantity", "damaged_quantity"]] = df[["sold_quantity", "returned_quantity", "damaged_quantity"]].replace(["unknown"," ",0], np.nan)

In [ ]:
print(df["sold_quantity"].value_counts(dropna=False))

In [ ]:
print(df["returned_quantity"].value_counts(dropna=False))

In [ ]:
print(df["damaged_quantity"].value_counts(dropna=False))

### Replace the NaN with the average value of the trimester for The column "production_quantity"

In [ ]:
import numpy as np
import pandas as pd

# Columns to be filled
columns_to_fill = ['sold_quantity', 'returned_quantity', 'damaged_quantity']

for col in columns_to_fill:
    for trimester_df in [df_trimester1, df_trimester2, df_trimester3, df_trimester4 ,df]:
        # Ensure 'production_quantity' and the current column are numeric
        trimester_df['production_quantity'] = pd.to_numeric(trimester_df['production_quantity'], errors='coerce')
        trimester_df[col] = pd.to_numeric(trimester_df[col], errors='coerce')

        # Compute the mean ratio before replacing values
        ratio_mean = (trimester_df[col] / trimester_df['production_quantity']).mean(skipna=True)
        
        # Check for any zero production quantity to avoid division by zero
        if np.any(trimester_df['production_quantity'] == 0):
            print(f"Warning: Zero production quantities found in {col} for trimester.")

        # Replace NaN and 0 values with the computed ratio
        mask = (trimester_df[col].isna()) | (trimester_df[col] == 0)
        trimester_df.loc[mask, col] = (
            (trimester_df.loc[mask, 'production_quantity'] * ratio_mean).astype(int)
        )

        # Convert to integer after all replacements (if necessary)
        trimester_df[col] = trimester_df[col].astype(int)

# Checking value counts after replacements
print(df["sold_quantity"].value_counts(dropna=False))


### Create unsold product column

In [ ]:
df['unsold_quantity'] = df['production_quantity'] - (df['damaged_quantity'] + df['returned_quantity'] + df['sold_quantity'])
df_trimester1['unsold_quantity'] = df_trimester1['production_quantity'] - (df_trimester1['damaged_quantity'] + df_trimester1['returned_quantity'] + df_trimester1['sold_quantity'])
df_trimester2['unsold_quantity'] = df_trimester2['production_quantity'] - (df_trimester2['damaged_quantity'] + df_trimester2['returned_quantity'] + df_trimester2['sold_quantity'])
df_trimester3['unsold_quantity'] = df_trimester3['production_quantity'] - (df_trimester3['damaged_quantity'] + df_trimester3['returned_quantity'] + df_trimester3['sold_quantity'])
df_trimester4['unsold_quantity'] = df_trimester4['production_quantity'] - (df_trimester4['damaged_quantity'] + df_trimester4['returned_quantity'] + df_trimester4['sold_quantity'])

### Check the new column

In [ ]:
df.head()

### Check the "product_name" values

In [ ]:
df["product_name"].value_counts()

### Fix incorrect Syntax 

In [ ]:
print(df.columns[df.columns.duplicated()])  # Check duplicate columns
print(df_trimester1.columns[df_trimester1.columns.duplicated()])

In [ ]:
df_trimester1.head()

In [ ]:
corrections = {
    "Yaourt Brassé Fraise Ba@ane 70g": "Yaourt Brassé Fraise Banane 70g",
    "Vit'up Max@Vanille 260g": "Vit'up Max Vanille 260g",
    "Yaourt aux Fruits Secs @00g": "Yaourt aux Fruits Secs 100g",
    "Yaourt aux Fruits Mixés Mangue 10@g": "Yaourt aux Fruits Mixés Mangue 100g",
    "Yaou@t aux Fruits Pêche Abricot 100g": "Yaourt aux Fruits Pêche Abricot 100g",
    "Yaourt Brassé @iscuit-Vanille 70g": "Yaourt Brassé Biscuit-Vanille 70g",
    "Yaourt Aromatisé@Vanille 100g": "Yaourt Aromatisé Vanille 100g",
    "Grecos Nature Su@ré": "Grecos Nature Sucré",
    "Vitupti Frais@ 100g": "Vitupti Fraise 100g",
    "Yaourt aux Fruits Pêche Abricot 10@g": "Yaourt aux Fruits Pêche Abricot 100g",
    "Yaourt Aromatise Fraise 100g": "Yaourt Aromatisé Fraise 100g",
    "Yaourt Aromatise Vanille 100g": "Yaourt Aromatisé Vanille 100g",
    "Yaourt Aromatise Noix de coco 100g": "Yaourt Aromatisé Noix de coco 100g",
    "Yaourt Aromatise Banane 100g": "Yaourt Aromatisé Banane 100g",
    "vitup Vanille 160g": "Vit'up Vanille 160g",
    "vitup Max Vanille 260g": "Vit'up Max Vanille 260g",
    "vitup Pinacolada 160g": "Vit'up Pinacolada 160g",
    "vitup Pinacolada 260g": "Vit'up Pinacolada 260g",
    "vitup Fraise 160g": "Vit'up Fraise 160g",
    "vitup Max Fraise 260g": "Vit'up Max Fraise 260g"
}
df["product_name"] = df["product_name"].replace(corrections)
df_trimester1["product_name"] = df_trimester1["product_name"].replace(corrections)
df_trimester2["product_name"] = df_trimester2["product_name"].replace(corrections)
df_trimester3["product_name"] = df_trimester3["product_name"].replace(corrections)
df_trimester4["product_name"] = df_trimester4["product_name"].replace(corrections)


In [ ]:
df["product_name"].value_counts()

### Remove Erroneous Values

After visiting the factory and gathering information on how the production machines operate, we found that for the most common yogurt type, there are four machines, each with a maximum production capacity of 80,000 units. Therefore, we must remove any production line with a quantity exceeding 360,000 units.

In [ ]:
df_trimester1.head()


In [ ]:
df.describe()

In [ ]:
sales_data = {
    "Trimester": ["T1", "T2", "T3", "T4"],
    "Total Sales": [
        df_trimester1["sold_quantity"].sum(),
        df_trimester2["sold_quantity"].sum(),
        df_trimester3["sold_quantity"].sum(),  # ✅ Fixed incorrect trimester reference
        df_trimester4["sold_quantity"].sum()   # ✅ Fixed incorrect trimester reference
    ]
}

# Convert to DataFrame
df_sales = pd.DataFrame(sales_data)

# Create interactive Plotly bar chart
fig = px.bar(
    df_sales, 
    x="Trimester", 
    y="Total Sales", 
    text=df_sales["Total Sales"].apply(lambda x: f"{x/1e6:.1f}M"),  # Formatting in millions
    labels={"Total Sales": "Total Sold Quantity"},
    title="Total Sales per Trimester",
    color="Total Sales",  # Color based on sales amount
    color_continuous_scale="Blues"  # Custom color scale
)

# Format y-axis to display in millions
fig.update_yaxes(
    tickformat=".1s", 
    title_text="Total Sold Quantity"
)

# Adjust text label position
fig.update_traces(textposition="outside")

# Show the figure
fig.show()

In [ ]:
def get_top5(df, trimester_name):
    top5 = df.groupby("product_name")["sold_quantity"].sum().nlargest(5).reset_index()
    top5["trimester"] = trimester_name  # Add trimester name column
    return top5

# Get top 5 products from each trimester
df_top5_s1 = get_top5(df_trimester1, "T1")
df_top5_s2 = get_top5(df_trimester2, "T2")
df_top5_s3 = get_top5(df_trimester3, "T3")
df_top5_s4 = get_top5(df_trimester4, "T4")

# Combine data
df_top5 = pd.concat([df_top5_s1, df_top5_s2, df_top5_s3, df_top5_s4])

# Create interactive Plotly bar chart
fig = px.bar(
    df_top5, 
    x="trimester", 
    y="sold_quantity", 
    color="product_name",  # Different color for each product
    text=df_top5["sold_quantity"].apply(lambda x: f"{x:,.0f}"),  # Formatting numbers
    labels={"sold_quantity": "Total Sold Quantity"},
    title="Top 5 Products by Sales in Each Trimester",
    barmode="group",  # Bars side-by-side
    color_discrete_sequence=px.colors.qualitative.Set1  # Custom color theme
)

# Format y-axis to display in millions (like `1.2M`)
fig.update_yaxes(
    tickformat=".1s", 
    title_text="Total Sold Quantity"
)

# Display text labels on bars
fig.update_traces(textposition="outside")

# Show the figure
fig.show()

In [ ]:
# Compute top 5 products by sales per quantity
df_product_sales = df.groupby("product_name").agg(
    total_sales=('sold_quantity', 'sum'),
    total_produced=('production_quantity', 'sum')
).reset_index()

df_product_sales['sales_per_quantity'] = df_product_sales['total_sales'] / df_product_sales['total_produced']
df_top5_products = df_product_sales.nlargest(5, "sales_per_quantity")

# Create an interactive Plotly bar chart
fig = px.bar(
    df_top5_products, 
    x="product_name", 
    y="sales_per_quantity", 
    text=df_top5_products["sales_per_quantity"].apply(lambda x: f"{x*100:.1f}%"),  # Show percentage
    labels={"sales_per_quantity": "Sales per Quantity Produced"},
    title="Top 5 Products by Sales-to-Quantity Ratio",
    color="sales_per_quantity",  # Color based on value
    color_continuous_scale="Viridis"  # Adjust color scale
)

# Format y-axis to display percentages
fig.update_yaxes(
    tickformat=".1%",  # Show in percentage format
    title_text="Sales per Quantity Produced"
)

# Adjust text label position
fig.update_traces(textposition="outside")

# Show the figure
fig.show()

## top 5 %sale rate for each trimester

In [ ]:

trimesters = {
    "Trimester 1": df_trimester1,
    "Trimester 2": df_trimester2,
    "Trimester 3 (Summer)": df_trimester3,
    "Trimester 4": df_trimester4
}

for trimester_name, df_trimester in trimesters.items():
    df_product_sales = df_trimester.groupby("product_name").agg(
        total_sales=('sold_quantity', 'sum'),
        total_produced=('production_quantity', 'sum')
    ).reset_index()

    df_product_sales['sales_per_quantity'] = df_product_sales['total_sales'] / df_product_sales['total_produced']
    df_top5_products = df_product_sales.nlargest(5, "sales_per_quantity")

    fig = px.bar(df_top5_products, x="product_name", y="sales_per_quantity", 
                 text=df_top5_products["sales_per_quantity"].apply(lambda x: f"{x*100:.1f}%"),
                 title=f"Top 5 Products - {trimester_name}",
                 labels={"sales_per_quantity": "Sales per Quantity Produced"},
                 color="sales_per_quantity", 
                 color_continuous_scale="Viridis")  # ✅ Use a valid colorscale
    
    fig.update_traces(textposition="outside")
    fig.show()


### The 5 Least Efficient Products per Trimester (Sales-to-Production Ratio)

In [ ]:
trimesters = {
    "Trimester 1": df_trimester1,
    "Trimester 2": df_trimester2,
    "Trimester 3 (Summer)": df_trimester3,
    "Trimester 4": df_trimester4
}

for trimester_name, df_trimester in trimesters.items():
    df_product_sales = df_trimester.groupby("product_name").agg(
        total_sales=('sold_quantity', 'sum'),
        total_produced=('production_quantity', 'sum')
    ).reset_index()

    df_product_sales['sales_per_quantity'] = df_product_sales['total_sales'] / df_product_sales['total_produced']
    
    # Get the bottom 5 products instead of the top 5
    df_bottom5_products = df_product_sales.nsmallest(5, "sales_per_quantity")

    fig = px.bar(df_bottom5_products, x="product_name", y="sales_per_quantity", 
                 text=df_bottom5_products["sales_per_quantity"].apply(lambda x: f"{x*100:.1f}%"),
                 title=f"Bottom 5 Products - {trimester_name}",
                 labels={"sales_per_quantity": "Sales per Quantity Produced"},
                 color="sales_per_quantity", 
                 color_continuous_scale="Viridis")  

    fig.update_traces(textposition="outside")
    fig.show()


In [ ]:
df_unsold = df.groupby("product_name")["unsold_quantity"].sum().reset_index()
df_top_unsold = df_unsold.nlargest(5, "unsold_quantity")

# Create bar plot
fig = px.bar(df_top_unsold, x="product_name", y="unsold_quantity",
             title="Top 5 Most Unsold Products",
             labels={"unsold_quantity": "Total Unsold Quantity"},
             color="unsold_quantity", color_continuous_scale="Reds")

fig.show()


In [ ]:
df_unsold_ratio = df.groupby("product_name").agg(
    total_unsold=("unsold_quantity", "sum"),
    total_produced=("production_quantity", "sum")
).reset_index()

# Prevent division by zero
df_unsold_ratio["total_produced"].replace(0, float("nan"), inplace=True)

# Calculate unsold-to-production ratio
df_unsold_ratio["unsold_ratio"] = df_unsold_ratio["total_unsold"] / df_unsold_ratio["total_produced"]

# Get the top 10 worst-performing products
df_top_unsold_ratio = df_unsold_ratio.nlargest(5, "unsold_ratio")

# Create bar plot
fig = px.bar(df_top_unsold_ratio, x="product_name", y="unsold_ratio",
             title="Top 5 Products with Highest Unsold-to-Produced Ratio",
             labels={"unsold_ratio": "Unsold / Produced Ratio"},
             color="unsold_ratio", color_continuous_scale="Oranges")

fig.show()

### Add Revenue Column (Price * sold quantity)

In [ ]:
#replace ',' with '.' for parsing to float
for df_trimester in [df, df_trimester1, df_trimester2, df_trimester3, df_trimester4]:
    df_trimester['price'] = df_trimester['price'].replace({',': '.'}, regex=True)
    df_trimester['price'] = pd.to_numeric(df_trimester['price'], errors='raise')
    df_trimester['revenue'] = df_trimester['price'] * df_trimester['sold_quantity']
df["revenue"] = df["price"] * df["sold_quantity"]
df_trimester1["revenue"] = df_trimester1["price"] * df_trimester1["sold_quantity"]
df_trimester2["revenue"] = df_trimester2["price"] * df_trimester2["sold_quantity"]
df_trimester3["revenue"] = df_trimester3["price"] * df_trimester3["sold_quantity"]
df_trimester4["revenue"] = df_trimester4["price"] * df_trimester4["sold_quantity"]


In [ ]:
df_trimester4["revenue"].value_counts(dropna=False)

In [ ]:
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
df_trimester1['revenue'] = pd.to_numeric(df_trimester1['revenue'], errors='coerce')
df_trimester2['revenue'] = pd.to_numeric(df_trimester2['revenue'], errors='coerce')
df_trimester3['revenue'] = pd.to_numeric(df_trimester3['revenue'], errors='coerce')
df_trimester4['revenue'] = pd.to_numeric(df_trimester4['revenue'], errors='coerce')


In [ ]:
df_revenue = df.groupby("product_name")["revenue"].sum().reset_index()

fig = px.bar(df_revenue.nlargest(5, "revenue"), x="product_name", y="revenue",
             title="Top 5 Products by Revenue",
             labels={"revenue": "Total Revenue"},
             color="revenue", color_continuous_scale="Blues")

fig.show()


In [ ]:
df_bottom5_products_by_revenue = df.nsmallest(5, "revenue")

# Plotting the bottom 5 products by revenue
fig = px.bar(df_bottom5_products_by_revenue, x="product_name", y="revenue", 
             title="Bottom 5 Products by Revenue",
             labels={"revenue": "Total Revenue"},
             color="revenue", color_continuous_scale="Viridis")

fig.show()

In [ ]:
df_trimester2.head()

In [ ]:


for trimester_name, df_trimester in trimesters.items():
    # Grouping by product_name and summing the revenue directly
    df_product_sales = df_trimester.groupby("product_name").agg(
        revenue=('revenue', 'sum')
    ).reset_index()

    # Get the top 5 products by revenue
    df_top5_products = df_product_sales.nlargest(5, "revenue")

    # Plot for top 5 products by revenue
    fig_top5 = px.bar(df_top5_products, x="product_name", y="revenue", 
                      text=df_top5_products["revenue"].apply(lambda x: f"{x/1e6:,.2f}M"),
                      title=f"Top 5 Products by Revenue - {trimester_name}",
                      labels={"revenue": "Total Revenue"},
                      color="revenue", color_continuous_scale="Viridis",
                     height=600)
    fig_top5.update_traces(textposition="outside")
    fig_top5.show()


In [ ]:

for trimester_name, df_trimester in trimesters.items():
 # Select bottom 5 products by revenue
    df_bottom5_products = df_product_sales.nsmallest(5, "revenue")

    # Plot for bottom 5 products by revenue
    fig_bottom5 = px.bar(df_bottom5_products, x="product_name", y="revenue", 
                         text=df_bottom5_products["revenue"].apply(lambda x: f"{x/1e6:,.2f}M"),
                         title=f"Bottom 5 Products by Revenue - {trimester_name}",
                         labels={"revenue": "Total Revenue"},
                         color="revenue", color_continuous_scale="Viridis",
                         height=600)
    
    fig_bottom5.update_traces(textposition="outside")
    fig_bottom5.show()  # This will show the plot for the current trimester